In [1]:
### THIS SCRIPT CLEANS THE COLLECTED SEP ###   

In [2]:
import pymongo 
import requests
import json
import pandas as pd
import re

from tqdm import tqdm
from bson.objectid import ObjectId
from pprint import pprint

import lib_sepinpho as sep

##### INIT GLOBAL VARIABLES#####

#init Mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

#create database
db = client.visualizing_sep

In [4]:
##### Cleaning Sep_Entries ########
##### Fall 2020 Edition: Completed 10.1.2020 ########
##### Spring 2021 Edition: No Updates ########
##### Summer & Fall 2021 Edition: No Updates ########
##### Winter Edition: 1.3.2022 ########

# #1. Check for entries with InPhO apis.

# collection_to_clean = db.sep_entries_dupe

# sep_no_inpho = pd.DataFrame(collection_to_clean.find( filter={'inpho_api': 'Error: No InPhO entry'},
#                                          projection = ['page_url','title'],
#                                          sort=[('title',1)]
#                                         ))
# #export to csv to made edits in this file
# sep_no_inpho.to_csv('noinpho.csv')


In [18]:
# read the values to update from the CSV and batch update the entries
##### Winter Edition: 1.3.2022 ########

# collection_to_update = db.sep_entries

# #import cleaned data
# inpho_updates = pd.read_csv('noinpho.csv')

# for index,row in tqdm(inpho_updates.iterrows()):
#     title = row['title']
#     api_endpoint = row['api_endpoint']
#     json_type = row['json_type']

#     sep.update_sep_json(title,api_endpoint, json_type, collection_to_update)



In [19]:
##### Cleaning Sep_Entries ########
##### Summer: Completed 7.16.2020 ########
##### Summer: Completed 10.1.2020 ########
##### Winter Edition: 1.3.2022 (none) ########

#2. Check for duplicate InPhO JSON data.

collection_to_clean = db.sep_entries
result = list(collection_to_clean.aggregate([
                                {
                                    '$group' : { '_id' : "$inpho_json.ID",
                                    'count': { '$sum': 1 }
                                    }
                                },
                                {
                                    '$match': { 'count': {'$ne': 1 }}
                                }]))

pprint(result)

[{'_id': None, 'count': 17}]


In [7]:
##### Winter Edition: 1.3.2022 ########

#Find any article that doesn't have domain info

# collection_to_update = db.sep_entries_dupe
# no_domains = pd.DataFrame(collection_to_update.find(
#     {'primary_domain': {'$exists': False}},
#     projection={'page_url':1, 'title':1, '_id':0}
# ))
# #export to csv to made edits in this file
# no_domains.to_csv('no_domains.csv')

In [21]:
#update articles in no_domains with new domain data  
#Fall Update: Completed 10.1.2020#
#SPring 2021 Update: Completed 4.3.2021
# Summer and Fall 2021 Updates: Completed 9.27.2021
# Winter 2021: 1.3.2022


# collection_to_update = db.sep_entries

# #import cleaned data
# nodmain_updates = pd.read_csv('no_domains.csv')

# for index,row in tqdm(nodmain_updates.iterrows()):
#     page_url = row['page_url']
#     primary_domain = row['primarydomain']
#     domain_list = row['domainlist']

#     sep.update_domain_info(page_url, domain_list, primary_domain, collection_to_update)

In [22]:
#now test again for no domain info

collection_to_update = db.sep_entries
no_domains = list(collection_to_update.find(
    {'primary_domain': {'$exists': False}},
    projection={'page_url':1, 'title':1, '_id':0}
))

pprint(no_domains)

[]


In [23]:
#Find any article that doesn't have inpho tag info

collection_to_update = db.sep_entries
no_domains = pd.DataFrame(collection_to_update.find(
    {'inpho_json': {'$exists': False}},
    projection={'page_url':1, 'title':1, '_id':0}
))
#export to csv to made edits in this file
no_domains.to_csv('no_inphosjon.csv')

In [25]:
#test for and remove editors notes
#Spring 2021 Update: I manually updated 8 articles in compass to remove the Editors Note from the preamble
#winter 2021: 1.3.2022

collection_to_update = db.sep_entries

editors_notes = list(collection_to_update.find(
    {'preamble_text': {'$regex': '^\['}},
    projection={'page_url':1, 'title':1, 'preamble_text':1, '_id':0}
))
pprint(len(editors_notes))
# pprint(editors_notes)

for article in editors_notes:
    print(article['page_url'])

0
